# How to create an LGST-only report
This example shows how you can create a HTML report from just the results of running *linear GST* (LGST).  This can be useful when you want to get a rough estimate of your gates quickly, as LGST is takes substantially less data and computation time compared with long-sequence GST.  This example is modeled after Tutorial 0.

In [1]:
#Make print statements compatible with Python 2 and 3
from __future__ import print_function

#Import the pygsti module (always do this) and the standard XYI model
import pygsti
from pygsti.construction import std1Q_XYI

# 1) get the target Model
target_model = std1Q_XYI.target_model

# 2) get the building blocks needed to specify which operation sequences are needed
prep_fiducials, meas_fiducials = std1Q_XYI.prepStrs, std1Q_XYI.effectStrs

# 3) generate "fake" data from a depolarized version of target_model
mdl_datagen = target_model.depolarize(op_noise=0.1, spam_noise=0.001)
listOfExperiments = pygsti.construction.list_lgst_circuits(
                        prep_fiducials, meas_fiducials,target_model)

ds = pygsti.construction.generate_fake_data(mdl_datagen, listOfExperiments, nSamples=1000,
                                            sampleError="binomial", seed=1234)

#Note: from listOfExperiments we can also create an empty dataset file
# which has columns of zeros where actual data should go. 
pygsti.io.write_empty_dataset("example_files/LGSTReportDataTemplate.txt", listOfExperiments,
                              "## Columns = 0 count, 1 count")
# After replacing the zeros with actual data, the data set can be 
# loaded back into pyGSTi using the line below and used in the rest
# of this tutorial. 
#ds = pygsti.io.load_dataset("example_files/LGSTReportDataTemplate.txt")

print("Only %d sequences are required!" % len(listOfExperiments))

Only 92 sequences are required!


In [2]:
#Run LGST and create a report
# You can also eliminate gauge optimization step by setting gaugeOptParams=False
results = pygsti.do_linear_gst(ds, target_model, prep_fiducials, meas_fiducials)

pygsti.report.create_standard_report(results, filename="example_files/LGSTonlyReport",
                                    title="LGST-only Example Report", verbosity=2)

--- Circuit Creation ---
   92 sequences created
   Dataset has 92 entries: 92 utilized, 0 requested sequences were missing
--- LGST ---
  Singular values of I_tilde (truncating to first 4 of 6) = 
  4.244089943192679
  1.1594632778409208
  0.9651516670737965
  0.9297628363691268
  0.049256811347238104
  0.025150658372136828
  
  Singular values of target I_tilde (truncating to first 4 of 6) = 
  4.242640687119286
  1.414213562373096
  1.4142135623730956
  1.4142135623730954
  2.5038933168948026e-16
  2.023452063009528e-16
  
  -- Adding Gauge Optimized (go0) --
*** Creating workspace ***
*** Generating switchboard ***
Found standard clifford compilation from std1Q_XYI
*** Generating tables ***
  targetSpamBriefTable                          took 0.472459 seconds
  targetGatesBoxTable                           took 0.136802 seconds
  datasetOverviewTable                          took 0.002995 seconds
  bestGatesetSpamParametersTable                took 0.000741 seconds
  bestGatesetSpa

/Users/enielse/research/pyGSTi/packages/pygsti/extras/rb/theory.py:200: UserWarning:

Output may be unreliable because the model is not approximately trace-preserving.



  bestGatesVsTargetTable_gv                     took 0.240329 seconds
  bestGatesVsTargetTable_gvgerms                took 0.031975 seconds
  bestGatesVsTargetTable_gi                     took 0.015935 seconds
  bestGatesVsTargetTable_gigerms                took 0.004695 seconds
  bestGatesVsTargetTable_sum                    took 0.199279 seconds
  bestGatesetErrGenBoxTable                     took 0.638943 seconds
  metadataTable                                 took 0.000913 seconds
  stdoutBlock                                   took 0.000282 seconds
  profilerTable                                 took 0.000383 seconds
  softwareEnvTable                              took 0.032941 seconds
  exampleTable                                  took 0.048424 seconds
  singleMetricTable_gv                          took 0.274536 seconds
  singleMetricTable_gi                          took 0.016795 seconds
  fiducialListTable                             took 0.000978 seconds
  prepStrListTable  

Click to open the file [example_files/LGSTonlyReport/main.html](example_files/LGSTonlyReport/main.html) in your browser to view the report.